# Trip Advisor hotel review sentiments prediction and review rating prediction

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Load all the required libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import string
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
sns.set_style('darkgrid')

from wordcloud import WordCloud,STOPWORDS
stopwords = list(STOPWORDS)
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score 
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer as CVTZ

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer



def set_seed(seed=31415):
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

def RMSE(Y,YHAT):
    return np.sqrt(mean_squared_error(Y,YHAT))

plt.rc('figure',figsize=(20,11))

## Load the dataset

In [ ]:
data = pd.read_csv('../input/trip-advisor-hotel-reviews/tripadvisor_hotel_reviews.csv')
data.head()

## Check for null values

In [ ]:
# checking for null values

data.isnull().sum()

## Data cleaning

In [ ]:
import re

def  clean_text(text):
    """
    Fuction to clean the text data
    * symbols
    * change to lower_case
    """
    text = text.str.lower()
    text = text.apply(lambda T: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", T))  
        
    return text

In [ ]:
data['Review']= clean_text(data['Review'])

In [ ]:
data['Review'] = data['Review'].str.replace('#','')

In [ ]:
data.head()

## Count of each rating

In [ ]:
data.Rating.value_counts()

In [ ]:
sns.countplot(data=data,x='Rating', palette="Set3")

Its clearly visible that the target variable(Rating) is not balanced, as we have huge difference in rating 1 and 5. So we will be using some sampling technique to balance these classes.

## Getting the number of words in each review

In [ ]:
## Getting the number of words by splitting them by a space
words_per_review = [len(x.split(" ")) for x in data['Review']]
sns.distplot(words_per_review,fit=norm, kde=False)

## Wordcloud of most common words

In [ ]:
def wordCloud_generator(data):
    wordcloud = WordCloud(width = 800, height = 800,
                          background_color ='black',
                          min_font_size = 10,
                          colormap='Pastel1'
                         ).generate(" ".join(data.values))
    # plot the WordCloud image                        
    plt.figure(figsize = (10, 10), facecolor = None) 
    plt.imshow(wordcloud, interpolation='bilinear') 
    plt.axis("off") 
    plt.tight_layout(pad = 0) 
    plt.title("Most common words in Reviews",fontsize=30)
    plt.show() 
    
wordCloud_generator(data['Review'])

## Text preprocessing

Now let us preprocess Reviews using some NLP tchniques like:
1. converting to lowercase
2. Removing Punctuation
3. Removing stopwords
4. Stemming
5. Lemmatization

In [ ]:
punc=string.punctuation

stop_words = set(stopwords.words('english'))

stemmer = PorterStemmer()

lemmatizer = WordNetLemmatizer()

def data_preprocessing(txt):
    
    #converting to lowercase
    txt=txt.lower()
    
    #Removing Punctuation
    txt="".join([x for x in txt if x not in punc])
    
    #Removing stopwords
    txt=" ".join([word for word in str(txt).split() if word not in stop_words])
    
    #Stemming
    txt = " ".join([stemmer.stem(word) for word in txt.split()])
    
    #Lemmatization
    txt = " ".join([lemmatizer.lemmatize(word) for word in txt.split()])

    return txt

data['text'] = data['Review'].apply(data_preprocessing)


In [ ]:
data

## Vectorizing the input text

Now we will vectorise the ratings using TF-IDF scores and we will use ```toarray()``` to convert resultant sparse matrix to dense matrix. 

In [ ]:
### Creating a python object of the class TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=400, min_df=0.05, max_df=0.9)
tfidf = tfidfconverter.fit_transform(data['text']).toarray()

## Divide into training and test sets

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(tfidf,data['Rating'],test_size=0.2,random_state=42)

## Upsampling using SMOTE


Since our target variable is not balanced, we will use SMOTE algorithm to upsample the minority classes.

In [ ]:
#-----Upsampling----
from sklearn.utils import resample
from collections import Counter

print("Before Upsampling:-")
print(Counter(y_train))


# Let's use SMOTE to oversample
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train,y_train)

print("After Upsampling:-")
print(Counter(y_train))

## Model building

### 1. Naive Bayes

In [ ]:
# training a Naive Bayes classifier 
#very fast
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB().fit(X_train, y_train) 

y_pred_NB=mnb.predict(X_test)

print("Accuracy of Multinominal Naive Balyes:",accuracy_score(y_test, y_pred_NB))
print(classification_report(y_pred_NB,y_test))

### 2. Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression().fit(X_train, y_train)

y_pred_lr=logreg.predict(X_test)

print("Accuracy of Logistic Regression:",accuracy_score(y_test, y_pred_lr))
print(classification_report(y_pred_lr,y_test))

### 3. Decision Tree

In [ ]:
des_tree = DecisionTreeClassifier().fit(X_train, y_train)

y_pred_dt=des_tree.predict(X_test)

print("Accuracy of Decision Tree Classifier:",accuracy_score(y_test, y_pred_dt))
print(classification_report(y_pred_dt,y_test))

### 4. Random Forest

In [ ]:
rf = RandomForestClassifier().fit(X_train, y_train)

y_pred_rf=rf.predict(X_test)

print("Accuracy of Random Forest Classifier:",accuracy_score(y_test, y_pred_rf))
print(classification_report(y_pred_rf,y_test))

### 5. XGBoost

In [ ]:
#takes huge amount of time to execute
import xgboost as xgb

xgboost_clf = xgb.XGBClassifier().fit(X_train, y_train)

y_pred_xgb=xgboost_clf.predict(X_test)

print("Accuracy of XGBoost Classifier:",accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_pred_xgb,y_test))

### 6. k-Nearest Neighbours(KNN)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier 

knn = KNeighborsClassifier(n_neighbors = 5).fit(X_train, y_train)

y_pred_knn=knn.predict(X_test)

print("Accuracy of k-nearest neighbours Classifier:",accuracy_score(y_test, y_pred_knn))
print(classification_report(y_pred_knn,y_test))

### 7. Support Vector Machines(SVM)

In [ ]:
from sklearn.svm import SVC

svc = SVC()

y_pred_svm=knn.predict(X_test)

print("Accuracy of SVM Classifier:",accuracy_score(y_test, y_pred_svm))
print(classification_report(y_pred_svm,y_test))

We can see that Naive Bayes, Logistic Regression and Random forest are giving us maximum accuracy, In the next version I'll be fine tuning these algorithms using some hyperparameter optimization techniques and will use word embeddings like Word2Vec and Glove or even BERT to encode my input data.

### To be continued...